# Introducción

En este notebook se procesarán los videos y se extraerán las caracteristicas de los clips usando el modelo preentrenado X3D


In [5]:
import torch
import torchvision.transforms as transforms
import torchvision.models.video as models
import cv2
import numpy as np
from PIL import Image
from imblearn.over_sampling import SMOTE
import re
import os
import json
import pickle
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import torch
import numpy as np
from torchvision import transforms
from fvcore.common.config import CfgNode as CN
import torch 
import numpy as np
from tqdm import tqdm


In [6]:
# Funcion para agrupar la ruta de los videos por accion   y ordenar el diccionario según la acción


def group_videos_by_action(root_path, video_extensions=None):
    if video_extensions is None:
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']

    grouped_videos = defaultdict(list)

    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in video_extensions):
                action_name = os.path.basename(dirpath) 
                file_path = os.path.join(dirpath, file)
                grouped_videos[action_name].append(file_path)
    
    sorted_actions = sorted(
        grouped_videos.items(), 
        key=lambda x: int(x[0].split('_')[1])  # Extraer número después de "action_"
    )
    return grouped_videos



def ordenar_diccionario(input_dict):

    # Ordenamos las claves extrayendo el número
    sorted_keys = sorted(input_dict.keys(), key=lambda x: int(re.search(r'\d+', x).group()))
    
    # Creamos un nuevo defaultdict ordenado
    sorted_dict = defaultdict(list, {key: input_dict[key] for key in sorted_keys})

    return sorted_dict

root_path = "F:/data/mvfouls/augmented_videos_severity"
videos_path = group_videos_by_action(root_path = root_path)
sorted_keys = ordenar_diccionario(videos_path)


In [7]:
sorted_keys

defaultdict(list,
            {'action_2916': ['F:/data/mvfouls/augmented_videos_severity\\action_2916\\clip_0_augmented.mp4',
              'F:/data/mvfouls/augmented_videos_severity\\action_2916\\clip_1_augmented.mp4',
              'F:/data/mvfouls/augmented_videos_severity\\action_2916\\clip_2_augmented.mp4'],
             'action_2917': ['F:/data/mvfouls/augmented_videos_severity\\action_2917\\clip_0_augmented.mp4',
              'F:/data/mvfouls/augmented_videos_severity\\action_2917\\clip_1_augmented.mp4'],
             'action_2918': ['F:/data/mvfouls/augmented_videos_severity\\action_2918\\clip_0_augmented.mp4',
              'F:/data/mvfouls/augmented_videos_severity\\action_2918\\clip_1_augmented.mp4'],
             'action_2919': ['F:/data/mvfouls/augmented_videos_severity\\action_2919\\clip_0_augmented.mp4',
              'F:/data/mvfouls/augmented_videos_severity\\action_2919\\clip_1_augmented.mp4',
              'F:/data/mvfouls/augmented_videos_severity\\action_2919\\cl

In [8]:
#Dividimos las acciones en grupos para facilitar su procesamiento
def save_dict_chunks(input_dict, chunk_size, save_path, format='npy'):
    keys = list(input_dict.keys())
    for i in range(0, len(keys), chunk_size):
        chunk = {key: input_dict[key] for key in keys[i:i + chunk_size]}
        file_name = f"{save_path}_chunk_{i // chunk_size}.{format}"
        
        if format == 'npy':
            np.save(file_name, chunk)
        elif format == 'pkl':
            with open(file_name, 'wb') as f:
                pickle.dump(chunk, f)
        else:
            raise ValueError("Formato no soportado. Usa 'npy' o 'pkl'.")
        print(f"Guardado: {file_name}")



save_dict_chunks(
    sorted_keys,
    chunk_size=500,  # Número de elementos por archivo
    save_path='F:/data/mvfouls/augmented_videos_severity',
    format='npy' 
)


Guardado: F:/data/mvfouls/augmented_videos_severity_chunk_0.npy
Guardado: F:/data/mvfouls/augmented_videos_severity_chunk_1.npy
Guardado: F:/data/mvfouls/augmented_videos_severity_chunk_2.npy


In [9]:
#Cargamos el chunk a procesar
chunk = np.load('F:/data/mvfouls/augmented_videos_severity_chunk_2.npy', allow_pickle=True).item()

In [10]:

# Función para cargar el modelo preentrenado X3D-S
def load_x3d_model():
    model_name = 'x3d_s'
    model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
    model.eval()  # Poner el modelo en modo de evaluación
    return model

# Función para extraer frames de un video en el formato adecuado
def video_to_frames(video_path, frame_size=(224, 224), num_frames=32):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error al abrir el video: {video_path}")
        
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
        
        if len(frames) >= num_frames:
            break

    cap.release()
    
    if len(frames) == 0:
        raise ValueError(f"No se pudieron extraer frames del video: {video_path}")
    
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    frames = np.array(frames)
    frames = np.transpose(frames, (0, 3, 1, 2))
    
    return frames

# Función para extraer características del video utilizando el modelo X3D-S
def extract_features(video_path, model):
    frames = video_to_frames(video_path)  # Extraer frames del video
    
    frames = torch.tensor(frames).float() / 255.0  # Normalizar los valores de los frames
    frames = frames.unsqueeze(0)  # Agregar dimensión de batch
    frames = frames.permute(0, 2, 1, 3, 4)  # Cambiar el orden de las dimensiones a [batch_size, num_channels, num_frames, height, width]

    with torch.no_grad():
        features = model(frames)
    
    return features.cpu().numpy()

# Función principal para extraer características de varios videos
def extract_features_from_grouped_videos(grouped_videos, model):
    action_features = {}
    video_paths_all = []  # Lista global de todos los videos a procesar
    
    # Recopilar todos los caminos de los videos en una lista
    for action, video_paths in grouped_videos.items():
        video_paths_all.extend(video_paths)
    
    # Usamos tqdm para crear una barra de progreso global
    for video_path in tqdm(video_paths_all, desc="Procesando videos", unit="video"):
        action = video_path.split("\\")[-2]  # Obtener el nombre de la acción desde la ruta (puedes ajustarlo)
        
        # Crear un lugar para las características de cada video
        if action not in action_features:
            action_features[action] = []
        
        try:
            features = extract_features(video_path, model)
            action_features[action].append(features)
        except Exception as e:
            print(f"Error al procesar el video {video_path}: {e}")
    
    # Promediar las características de cada acción
    for action, clips in action_features.items():
        action_features[action] = np.mean(clips, axis=0)
    
    return action_features
# Cargar el modelo X3D-S
try:
    x3d_model = load_x3d_model()
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    x3d_model = None


# Extraer características de los videos
if x3d_model is not None:
    try:
        extracted_features = extract_features_from_grouped_videos(chunk, x3d_model)
        print(extracted_features)
    except Exception as e:
        print(f"Error al extraer características de los videos: {e}")
else:
    print("El modelo no se cargó correctamente, no se puede continuar con la extracción de características.")

Using cache found in C:\Users\juanm/.cache\torch\hub\facebookresearch_pytorchvideo_main
Procesando videos:   1%|▍                                                           | 3/369 [00:02<05:45,  1.06video/s]


KeyboardInterrupt: 

In [ ]:
# Guardar los resultados parciales
with open('results_augmented_severity_2.pkl', 'wb') as f:
    pickle.dump(extracted_features, f)  # Guarda el diccionario de resultados

In [ ]:
extracted_features

In [ ]:
#Funciones para juntar los resultados parciales en un único archivo

directory_path = "F:/MVFoulRecognition/features/X3D/train/severity"
prefix = "results_" # Prefijo para buscar los archivos 

files = []

for filename in os.listdir(directory_path):
    full_path = os.path.join(directory_path, filename)
    if os.path.isfile(full_path) and filename.startswith(prefix):  # Verifica si es un archivo y tiene el prefijo
        files.append(full_path)

print("Archivos encontrados:")
sorted_files = sorted(files, key=lambda x: int(x.split('_severity_')[-1].split('.pkl')[0]))
# Imprime la lista ordenada
for file in sorted_files:
    print(file)
    
combined_data = []
# Cargar y combinar los datos
for file in sorted_files:
    with open(file, 'rb') as f:
        data = pickle.load(f)  # Carga el archivo .pkl
        combined_data.append(data)  # Agrega el array a la lista

# Guardar el array combinado en un nuevo archivo .pkl
with open('train_augmented_combined.pkl', 'wb') as f:
    pickle.dump(combined_data, f)

print("Arrays combinados guardados en 'train_combined.pkl'")


In [ ]:
#Agrupamos en un mismo archivo los resultados de la extraccion de videos y videos aumentados

# Nombres de los archivos
archivo2 = "F:/MVFoulRecognition/features/X3D/train/severity/end/train_augmented_combined.pkl"
archivo1 = "F:/MVFoulRecognition/features/X3D/train/end/train_combined.pkl"
archivo_combinado = "train_combinados.pkl"

with open(archivo1, "rb") as f1, open(archivo2, "rb") as f2:
    datos1 = pickle.load(f1)
    datos2 = pickle.load(f2)

if isinstance(datos1, list) and isinstance(datos2, list):
    datos_combinados = datos1 + datos2  # Unir listas
elif isinstance(datos1, dict) and isinstance(datos2, dict):
    datos_combinados = {**datos1, **datos2}  # Unir diccionarios
else:
    raise TypeError("Los archivos tienen estructuras incompatibles")

# Guardar el archivo combinado
with open(archivo_combinado, "wb") as f:
    pickle.dump(datos_combinados, f)

print(f"Archivos combinados y guardados en '{archivo_combinado}'")


# A partir de aquí termina la parte de extracción de features y comienza el entrenamiento (podemos mirar otro notebook)

In [ ]:

with open('F:/MVFoulRecognition/features/X3D/train/sev/end/train_combinados.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_train = {k: v for dic in loaded_data for k, v in dic.items()}

# Salida: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
dic_ordenado_train = dict(sorted(resultado_train.items(), key=lambda x: int(x[0].split('_')[1])))

#print(dic_ordenado_train)
with open('F:/MVFoulRecognition/features/X3D/test/test_combined.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_test = {k: v for dic in loaded_data for k, v in dic.items()}

# Salida: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
dic_ordenado_test = dict(sorted(resultado_test.items(), key=lambda x: int(x[0].split('_')[1])))

In [ ]:
dic_ordenado_train.keys()

In [ ]:
def load_action_labels(json_path):
    """
    Carga las etiquetas de acción desde un archivo JSON.

    Args:
        json_path (str): Ruta al archivo JSON.

    Returns:
        dict: Diccionario donde las claves son nombres de acciones (ej. 'action_0')
              y los valores son las etiquetas (ej. 'Challenge', 'Tackling').
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    action_labels = {}
    for action_id, action_data in data['Actions'].items():
        label = action_data['Offence']  # Extraer la etiqueta de 'Action class'
        action_name = f"action_{action_id}"  # Formar el nombre de la acción
        action_labels[action_name] = label
    
    return action_labels



In [ ]:
ls = load_action_labels("F:/data/mvfouls/train/annotations.json")
ls = sorted(ls.items())
sorted_labels_train = sorted(ls, key=lambda x: int(x[0].split('_')[1]))

ls = load_action_labels("F:/data/mvfouls/test/annotations.json")
ls = sorted(ls.items())
sorted_labels_test = sorted(ls, key=lambda x: int(x[0].split('_')[1]))

In [ ]:
sorted_labels_train.extend([('action_' + str(i), 'No offence') for i in range(2916, 3945)])

# Verificar algunas entradas
print(sorted_labels_train[-10:])  # Últimos 10 elementos

# Crear un diccionario con claves numéricas del 0 al 3999
diccionario = {i: 0 for i in range(0, 3945)}

# Recorrer las claves en `res`
for r in dic_ordenado_train:
    # Extraer el número de la clave 'action_X'
    action = int(r.split("_")[1])  # Convertimos el número extraído a entero
    
    # Si el número extraído de `action_X` está en `diccionario`, lo marcamos como 1
    if action in diccionario:
        diccionario[action] = 1

# Mostrar el diccionario actualizado
diccionario_filtrado = {k: v for k, v in diccionario.items() if v == 0}
print(diccionario_filtrado)

In [ ]:
sorted_labels_train

In [ ]:
sorted_labels_train = [item for item in sorted_labels_train if item[0] != 'action_3266' and item[0] != 'action_3569' ]

for item in sorted_labels_train:
    if item[0] == 'action_3569':
        print(item)
        
# 1. Filtrar las acciones con etiqueta 'Offence'
offence_actions = [item for item in sorted_labels_train if item[1] == 'Offence']

# 2. Seleccionar las primeras mil acciones con etiqueta 'Offence'
# Aseguramos que hay al menos mil acciones con etiqueta 'Offence'
actions_to_remove = offence_actions[:1000]

# 3. Guardar estas mil acciones en una lista
actions_to_remove_list = actions_to_remove

sorted_labels_train = [item for item in sorted_labels_train if item not in actions_to_remove_list]

In [ ]:
for action in actions_to_remove_list:
    dic_ordenado_train.pop(action[0], None) 
# Ver el diccionario después de la eliminación
print("Diccionario después de eliminar las acciones:", dic_ordenado_train.keys())

In [ ]:
X_train  =np.array(list(dic_ordenado_train.values()))
labels_dic_train = dict(sorted_labels_train)
Y_train = np.array(list(labels_dic_train.values()))

X_test = np.array(list(dic_ordenado_test.values()))
labels_dict_test = dict(sorted_labels_test)
Y_test = np.array(list(labels_dict_test.values()))

print("Ejemplo de etiquetas en Y_train:", Y_train[:10])
print("Ejemplo de etiquetas en Y_valid:", Y_test[:10])

label_encoder = LabelEncoder()

# Ajustamos el codificador con TODAS las etiquetas posibles
label_encoder.fit(np.concatenate([Y_train, Y_test]))

# Transformamos las etiquetas de texto a números
Y_train_encoded = label_encoder.transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

print("Etiquetas originales:", np.unique(Y_train)) 
print("Etiquetas codificadas:", np.unique(Y_train_encoded))
print("Diccionario de conversión:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


In [ ]:
print(f"X_train type: {type(X_train)}, Y_train type: {type(Y_train)}")
print(f"X_valid type: {type(X_test)}, Y_valid type: {type(Y_test)}")

# Verificar las dimensiones
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_valid shape: {X_test.shape}, Y_valid shape: {Y_test.shape}")




In [ ]:

# Aplanar la segunda dimensión de X_train y X_valid
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)  # (301, 400)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


# Aplanar la segunda dimensión de X_train y X_valid
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)  # (301, 400)

print("X_train shape después de aplanar:", X_train_flattened.shape)
print("X_valid shape después de aplanar:", X_test_flattened.shape)

# # Aplicar SMOTE para balancear las clases
#smote = SMOTE(random_state=42)
#X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_flattened, Y_train_encoded)



# Crear el modelo
rf_model = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train_resampled, Y_train_resampled)

# Predecir en el conjunto de validación
Y_test_pred = rf_model.predict(X_test_flattened)

# Evaluar el rendimiento
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Random Forest Accuracy: {accuracy}")


In [ ]:
# Evaluar la precisión
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Precisión del modelo optimizado: {accuracy}")

# Evaluar el F1 score
f1 = f1_score(Y_test_encoded, Y_test_pred)
print(f"F1 Score del modelo optimizado: {f1}")

# Mostrar la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(Y_test_encoded, Y_test_pred))

# Mostrar el reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(Y_test_encoded, Y_test_pred))

In [ ]:
  # Crear y entrenar el modelo XGBoost
import xgboost as xgb
model = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train_resampled, Y_train_resampled)

# Evaluar el modelo
y_pred = model.predict(X_test_flattened)
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Precisión del modelo: {accuracy:.4f}")
# Evaluar el F1 score
f1 = f1_score(Y_test_encoded, Y_test_pred)
print(f"F1 Score del modelo optimizado: {f1}")

# Mostrar la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(Y_test_encoded, Y_test_pred))

# Mostrar el reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(Y_test_encoded, Y_test_pred))

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np

# Definir el espacio de búsqueda
param_dist = {
    'C': uniform(0.1, 10),  # C entre 0.1 y 10
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],  # Solo para kernel 'poly'
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

# Definir el modelo base
svc = SVC()

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=svc,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='f1_macro',
    cv=3,
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# Ajustar al conjunto de entrenamiento
random_search.fit(X_train_flattened, Y_train_encoded)

# Resultados
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Predecir en test
best_svm = random_search.best_estimator_
Y_test_pred = best_svm.predict(X_test_flattened)

# Evaluar
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test_encoded, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test_encoded, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test_encoded, Y_test_pred))



In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np


# Aplanar las dimensiones de los datos (como antes)
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)    # (301, 400)

# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_flattened, Y_train_encoded)

# Verifica las dimensiones de los datos balanceados
print(f"X_train después de SMOTE: {X_train_resampled.shape}")
print(f"Y_train después de SMOTE: {Y_train_resampled.shape}")


xgb_model = xgb.XGBClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'max_depth': [3, 5, 7],  # Profundidad máxima del árbol
    'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'subsample': [0.8, 1.0],  # Fracción de muestras que se usan para entrenar cada árbol
    'colsample_bytree': [0.8, 1.0],  # Fracción de columnas usadas para cada árbol
    'gamma': [0, 0.1, 0.2],  # Penalización para la complejidad del árbol
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train_flattened, Y_train_encoded)

print(f"Mejores parámetros con GridSearchCV: {grid_search.best_params_}")

# Parámetros a optimizar con RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(50, 200, 50),  # Número de árboles
    'max_depth': [3, 5, 7, 9],  # Profundidad máxima del árbol
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Tasa de aprendizaje
    'subsample': [0.7, 0.8, 0.9, 1.0],  # Fracción de muestras usadas para entrenar
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],  # Fracción de columnas para cada árbol
    'gamma': [0, 0.1, 0.2, 0.3],  # Regularización para árboles
}

random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train_flattened, Y_train_encoded)

print(f"Mejores parámetros con RandomizedSearchCV: {random_search.best_params_}")

# Usar los mejores parámetros encontrados
best_model = grid_search.best_estimator_  # o random_search.best_estimator_

# Hacer predicciones con el modelo optimizado
Y_test_pred = best_model.predict(X_test_flattened)

# Evaluar la precisión
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Precisión del modelo optimizado: {accuracy}")


# Evaluar el rendimiento
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"XGBoost Accuracy: {accuracy}")

# Mostrar la matriz de confusión y el reporte de clasificación
print("Matriz de Confusión:")
print(confusion_matrix(Y_test_encoded, Y_test_pred))

print("Reporte de Clasificación:")
print(classification_report(Y_test_encoded, Y_test_pred))


In [ ]:
with open("modelo_xgboost_x3d.pkl", "wb") as f:
    pickle.dump(best_model, f)

In [ ]:
# Aplanar las dimensiones de los datos (como antes)
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)    # (301, 400)

# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
#smote = SMOTE(random_state=42)
#X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_flattened, Y_train_encoded)
print(np.unique(Y_train, return_counts=True))


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {
    'C': uniform(0.1, 10),
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

random_search = RandomizedSearchCV(
    estimator=SVC(),
    param_distributions=param_dist,
    n_iter=30,  # Prueba solo 30 combinaciones
    scoring='f1_macro',
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=2
)

random_search.fit(X_train_flattened, Y_train_encoded)
# Definir el modelo SVM
svm = SVC()

# Realizar la búsqueda de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=3, scoring='f1_macro', n_jobs=-1, verbose=1)

# Ajustar el modelo a los datos
grid_search.fit(X_train_flattened, Y_train_encoded)

# Imprimir los mejores parámetros encontrados
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")

# Usar el modelo con los mejores parámetros encontrados
best_svm = grid_search.best_estimator_

# Realizar predicciones sobre el conjunto de prueba
Y_test_pred = best_svm.predict(X_test_flattened)

# Evaluar la precisión
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Precisión del modelo optimizado: {accuracy}")


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

Y_test_pred = best_svm.predict(X_test_flattened)

# Evaluar la precisión
accuracy = accuracy_score(Y_test_encoded, Y_test_pred)
print(f"Precisión del modelo optimizado: {accuracy}")

# Evaluar el F1 score
f1 = f1_score(Y_test_encoded, Y_test_pred)
print(f"F1 Score del modelo optimizado: {f1}")

# Mostrar la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(Y_test_encoded, Y_test_pred))

# Mostrar el reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(Y_test_encoded, Y_test_pred))

In [ ]:
from sklearn.model_selection import train_test_split

# Suponiendo que estos son tus datos completos
X_total = np.concatenate((X_train_flattened, X_test_flattened))
Y_total = np.concatenate((Y_train_encoded, Y_test_encoded))

# Nueva división estratificada
X_train, X_test, Y_train, Y_test = train_test_split(
    X_total, Y_total,
    test_size=0.2,         # o el porcentaje que quieras
    stratify=Y_total,      # 👈 esto asegura la misma proporción de clases
    random_state=42
)


from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np

# Definir el espacio de búsqueda
param_dist = {
    'C': uniform(0.1, 10),  # C entre 0.1 y 10
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],  # Solo para kernel 'poly'
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

# Definir el modelo base
svc = SVC()

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=svc,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='f1_macro',
    cv=3,
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# Ajustar al conjunto de entrenamiento
random_search.fit(X_train, Y_train)

# Resultados
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Predecir en test
best_svm = random_search.best_estimator_
Y_test_pred = best_svm.predict(X_test)

# Evaluar
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test, Y_test_pred))



In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np

# Suponiendo que estos son tus datos completos
X_total = np.concatenate((X_train_flattened, X_test_flattened))
Y_total = np.concatenate((Y_train_encoded, Y_test_encoded))

# Nueva división estratificada
X_train, X_test, Y_train, Y_test = train_test_split(
    X_total, Y_total,
    test_size=0.2,         # o el porcentaje que quieras
    stratify=Y_total,      # 👈 esto asegura la misma proporción de clases
    random_state=42
)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],  # Número de árboles
    'learning_rate': uniform(0.01, 0.2),         # Tasa de aprendizaje
    'max_depth': [3, 4, 5, 6, 7],                # Profundidad máxima de los árboles
    'min_child_weight': [1, 2, 3, 4],            # Peso mínimo de los hijos
    'subsample': uniform(0.6, 0.4),              # Submuestra de las instancias
    'colsample_bytree': uniform(0.6, 0.4),       # Submuestra de las columnas
    'gamma': [0, 0.1, 0.2, 0.3],                 # Penalización por complejidad
    'class_weight': ['balanced', None],          # Manejo del desbalance
}

# Definir el modelo XGBoost
xgb = XGBClassifier(random_state=42)

# Realizar Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='f1_macro',  # Usar F1 macro como métrica de optimización
    cv=3,  # Validación cruzada
    random_state=42,
    verbose=2,
    n_jobs=-1  # Para usar todos los núcleos del procesador
)

# Ajustar al conjunto de entrenamiento
random_search.fit(X_train, Y_train)

# Resultados de los mejores parámetros
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Usar el mejor modelo
best_xgb = random_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
Y_test_pred = best_xgb.predict(X_test)

# Evaluar el modelo
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test, Y_test_pred))


In [ ]:
import pickle

with open('best_svm_x3d.pkl', 'wb') as f:
    pickle.dump(best_svm, f)